In [1]:
%env JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
%env FLINK_CONF_DIR=/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/config

env: JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'
env: FLINK_CONF_DIR=/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/config


In [2]:
%load_ext streaming_jupyter_integrations.magics

Could not find scli config file at '/Users/soufiane/Workspace/agile/medical-dashboard/back-end-synthea-flink/.streaming_config.yml'. Will not load any secrets.
Set env variable JAVA_TOOL_OPTIONS='--add-opens=java.base/java.util=ALL-UNNAMED --add-opens=java.base/java.lang=ALL-UNNAMED'


In [3]:
%flink_connect --execution-target remote --remote-hostname localhost --remote-port 8081


remote environment has been created.


In [4]:
%%flink_execute_sql
CREATE TABLE patient (
  `id` STRING,
  `name` ROW(`family` STRING, `given` STRING ARRAY) ARRAY,
  `gender` STRING,
  `birthDate` STRING,
  `telecom` ROW ( `value` STRING ) ARRAY,
  `address` ROW (
    `line` STRING ARRAY,
    `city` STRING,
    `state` STRING,
    `postalCode` STRING,
    `country` STRING
  ) ARRAY,
  `maritalStatus` ROW (
    `text` STRING
  )
) WITH (
  'connector' = 'kafka',
  'topic' = 'Patient',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

Job starting...
Job started
Execution successful


In [5]:
%%flink_execute_sql
CREATE TABLE encounter (
  `id` STRING,
  `status` STRING,
  `type` ROW (
    `coding` ROW (
      `code` STRING,
      `display` STRING
    ) ARRAY  ) ARRAY,
  `subject` ROW (
    `reference` STRING,
    `display` STRING
  ),
  `participant` ROW (
    `individual` ROW (
      `reference` STRING,
      `display` STRING
    )
  ) ARRAY,
  `period` ROW (
    `start` STRING,
    `end` STRING
  ),
  `serviceProvider` ROW (
    `reference` STRING
  ) 
) WITH (
  'connector' = 'kafka',
  'topic' = 'Encounter',
  'properties.bootstrap.servers' = 'kafka-edge1:9092',
  'properties.group.id' = 'testGroup',
  'scan.startup.mode' = 'earliest-offset',
  'format' = 'json'
)

Job starting...
Job started
Execution successful


In [11]:
%%flink_execute_sql
CREATE TABLE practitioner_stats_psql (
   `id_practitioner` STRING,
    `average_age` INTEGER,
    `min_age` INTEGER,
    `max_age` INTEGER,
    `total_patients` INTEGER,
  PRIMARY KEY (`id_practitioner`) NOT ENFORCED
) WITH (
  'connector' = 'jdbc',
  'url' = 'jdbc:postgresql://ep-dark-sea-a20skpa3-pooler.eu-central-1.aws.neon.tech:5432/verceldb',
  'table-name' = 'public.practitioner_stats',
  'username' = 'default',
  'password' = 'tUyXvsaMq07d'
);

Job starting...
Job started
Execution successful


In [ ]:
%%flink_execute_sql
INSERT INTO practitioner_stats_psql
WITH patient_ages AS (
    SELECT 
        p.id,
        YEAR(CAST(NOW() AS DATE)) - YEAR(CAST(p.birthDate AS DATE)) - 
        CASE WHEN EXTRACT(MONTH FROM CAST(NOW() AS DATE)) < EXTRACT(MONTH FROM CAST(p.birthDate AS DATE)) OR (EXTRACT(MONTH FROM CAST(NOW() AS DATE)) = EXTRACT(MONTH FROM CAST(p.birthDate AS DATE)) AND EXTRACT(DAY FROM CAST(NOW() AS DATE)) < EXTRACT(DAY FROM CAST(p.birthDate AS DATE))) THEN 1 ELSE 0 END AS age
    FROM 
        encounter e
    JOIN 
        patient p ON REPLACE(
      e.subject.reference, 
        'urn:uuid:', 
        ''
        ) = p.id
),
practitioner_stats AS (
    SELECT 
        REPLACE(
      e.subject.reference, 
        'urn:uuid:', 
        ''
        )
        AS id_practitioner,
        AVG(pa.age) AS average_age,
        MIN(pa.age) AS min_age,
        MAX(pa.age) AS max_age,
        COUNT(DISTINCT pa.id) AS total_patients
    FROM 
        encounter e
    JOIN 
        patient_ages pa ON REPLACE(
      e.subject.reference, 
        'urn:uuid:', 
        ''
        ) = pa.id
    GROUP BY 
        REPLACE(
    e.participant[1].individual.reference, 
    'Practitioner?identifier=http://hl7.org/fhir/sid/us-npi|', 
    ''
  ),
    e.subject.reference
)
SELECT 
    ps.id_practitioner,
    AVG(ps.average_age) AS average_age,
    MIN(ps.min_age) AS min_age,
    MAX(ps.max_age) AS max_age,
    SUM(ps.total_patients) AS total_patients
FROM 
    practitioner_stats ps
GROUP BY 
    ps.id_practitioner;


This job runs in a background, please either wait or interrupt its execution before continuing


Job starting...
Job started
Pulling query results...
Results will be pulled from the job. You can interrupt any time to show partial results.
Execution result will bind to `execution_result` variable.


IntText(value=0, description='Loaded rows: ')

,id_practitioner,average_age,min_age,max_age,total_patients
0,e6181521-8f18-0384-01c8-3196bda965c1,68,68,68,1
1,e6181521-8f18-0384-01c8-3196bda965c1,68,68,68,1
2,e6181521-8f18-0384-01c8-3196bda965c1,68,68,68,2
3,e6181521-8f18-0384-01c8-3196bda965c1,68,68,68,2
4,e6181521-8f18-0384-01c8-3196bda965c1,68,68,68,3
...,...,...,...,...,...
206,c36b439f-5f1d-2380-0957-80badc95ce3f,35,35,35,2
207,c36b439f-5f1d-2380-0957-80badc95ce3f,35,35,35,3
208,ae61b742-fb9f-b28a-9fab-b8c04b2d83d2,23,23,23,1
209,ae61b742-fb9f-b28a-9fab-b8c04b2d83d2,23,23,23,1


Deployment cancellation in progress...
Job termination in progress...
Execution done
